In [1]:
import pandas as pd
import numpy as np
import extract_msg
import datetime
import os

from datetime import timedelta

In [2]:
#holidays
feriados = ('1 Jan', '21 Apr', '1 May', '20 Jun', '7 Sep', '12 Oct', '02 Nov', '15 Nov', '24 Dec', '25 Dec', '31 Dec')
# week days
DDS = ('Mon', 'Tue', 'Wed', 'Thu', 'Fri')

In [3]:
# return the overtime in minutes
# considering regular, nocturnal and weekends/holidays overtime:
def overtime(df):
    if df['sat'] == 0 and df['sun'] == 0 and df['holiday'] == 0 and '22:00' > df['time'] > '18:30':
        diff = datetime.datetime.strptime(df['time'], '%H:%M:%S') - datetime.datetime.strptime('18:30:00', '%H:%M:%S')
        diff = int(diff.total_seconds() / 60) * 1.7
    elif df['sat'] == 0 and df['sun'] == 0 and df['holiday'] == 0 and df['time'] > '22:00':
        diff = datetime.datetime.strptime(df['time'], '%H:%M:%S') - datetime.datetime.strptime('22:00:00', '%H:%M:%S')
        diff = int(diff.total_seconds() / 60) * 1.7 * 1.5
    elif df['sat'] == 0 and df['sun'] == 0 and df['holiday'] == 0 and df['time'] < '05:00':
        diff = datetime.datetime.strptime('05:00:00', '%H:%M:%S') - datetime.datetime.strptime(df['time'], '%H:%M:%S')
        diff = int(diff.total_seconds() / 60) * 1.7 * 1.5
    elif df['sat'] == 1 or df['sun'] == 1 or df['holiday'] == 1:
        diff = 60 * 2
    else:
        diff = 0
    return diff

## DISCLAIMER: the original version is much more complete, taking into account the brazilian labour law and conventions
###            this is a simplified version, as proof of consept.

In [4]:
path = 'testing emails/'
data_base_inp = input("Digite a data do protocolo da demanda (dd/mm/aaaa): ")
media_salarial = input("Digite a media salarial mensal: ")

Digite a data do protocolo da demanda (dd/mm/aaaa): 08/02/2019
Digite a media salarial mensal: 10000


In [5]:
## other optional inputs useful for calculating overtime:
#data_desl = input("Digite a data de desligamento da empresa (dd/mm/aaaa): ")
#fds = input("Digite a quantidade aproximada de horas trabalhadas por fim de semana: ")
#fer = input("Digite a quantidade aproximada de horas trabalhadas por feriado: ")
#valor_hora = input("Digite o valor da hora trabalhada: ")
#media_salario = ("Digite a media mensal do salario: ")

In [6]:
try:
    data_base_a = data_base_inp.split(sep='/')
    dia_db = int(data_base_a[0])
    mes_db = int(data_base_a[1])
    ano_db = int(data_base_a[2])
    ano_base = int(ano_db) - 5
    data_base = str(ano_base) + str(mes_db) + str(dia_db)
    print("Coletando informações...")
except :
    print("Digite a data no formato solicitado: dd/mm/aaaa")


Coletando informações...


In [7]:
# loops throw the path indicated to read all e-mails in it
filenames = [path+file for file in os.listdir(path) if file.endswith('.msg')]
filenames

['testing emails/2017_11_29_Lista de pendencias levantadas.msg',
 'testing emails/2017_12_06_RES Lista de pendencias.msg',
 'testing emails/2017_12_08_RES RES Lista de pendencias.msg',
 'testing emails/2017_12_27_RES RES RES Lista de pendencias.msg',
 'testing emails/2018_01_16_RES RES RES Lista de pendencias.msg',
 'testing emails/2018_01_19_RES RES RES RES Lista de pendencias.msg']

In [9]:
# extract information on files
msg_sender=[extract_msg.Message(file).sender for file in filenames]
msg_date=[extract_msg.Message(file).date for file in filenames]
msg_date, msg_sender

(['Wed, 29 Nov 2017 16:09:01 +0000',
  'Wed, 06 Dec 2017 18:48:01 +0000',
  'Fri, 08 Dec 2017 13:57:05 +0000',
  'Tue, 26 Dec 2017 10:34:25 +0000',
  'Tue, 16 Jan 2018 18:06:16 +0000',
  'Fri, 19 Jan 2018 11:48:11 +0000'],
 ['<gerencia@softwarelab.com.br>',
  '<gerencia@softwarelab.com.br>',
  '<gerencia@softwarelab.com.br>',
  '<gerencia@softwarelab.com.br>',
  '<gerencia@softwarelab.com.br>',
  '<gerencia@softwarelab.com.br>'])

In [ ]:
#some files might contains encoding errors, therefore, you might want to use a try and except line to keep it going:

#count_err=0
#error=[]
#for file in filenames :
#    try:
#        msg_sender += msg.sender
#        msg_date += msg.date
#        print(msg_date, msg_sender)
#    except:
#        count_err += 1
#        error += file
#        continue

In [10]:
week_day = [i[:3] for i in msg_date]
date = [i[5:16].strip() for i in msg_date]
time = [i[16:25].strip() for i in msg_date]

In [11]:
data_frame = pd.DataFrame()
data_frame['sender'] = msg_sender
data_frame['week day'] = week_day
data_frame['date'] = [' '.join(d.split()[:2]) for d in date]
data_frame['year'] = [d.split()[-1] for d in date]
data_frame['time'] = time

#inserting test examples
examples = {'sender': ['<gerencia@softwarelab.com.br>','<gerencia@softwarelab.com.br>','<gerencia@softwarelab.com.br>',
                       '<gerencia@softwarelab.com.br>', '<gerencia@softwarelab.com.br>','<gerencia@softwarelab.com.br>',
                       '<gerencia@softwarelab.com.br>','<gerencia@softwarelab.com.br>'],
            'week day': ['Sat','Sat','Sun', 'Sun', 'Sat', 'Tue', 'Thu', 'Thu'],
            'date': ['21 Dec','21 Dec','22 Dec','22 Dec', '7 Sep', '10 Dec', '12 Dec', '12 Dec'],
            'year': ['2019','2019','2019','2019','2019','2019','2019','2019'],
            'time': ['12:00:00', '12:30:00', '19:00:00', '22:40:00', '12:00:00', '03:40:00', '19:00:00', '23:40:00']
           }

data_frame_2 = pd.DataFrame.from_dict(examples)

data_frame = pd.concat([data_frame, data_frame_2])

In [13]:
data_frame['date_year'] = data_frame.apply(lambda x: x['date'] + ' ' + x['year'], axis=1)
data_frame['holiday'] = data_frame['date'].apply(lambda x: 1 if x in feriados else 0)
data_frame['sat'] = data_frame['week day'].apply(lambda x: 1 if x == 'Sat' else 0)
data_frame['sun'] = data_frame['week day'].apply(lambda x: 1 if x == 'Sun' else 0)
data_frame['overtime'] = data_frame.apply(overtime, axis=1)

#avoiding repeated dates
over_df = data_frame.groupby('date_year').agg({'overtime':max}).sort_values('overtime', ascending=False)
over_df

,overtime
date_year,
12 Dec 2019,255.0
10 Dec 2019,204.0
21 Dec 2019,120.0
22 Dec 2019,120.0
7 Sep 2019,120.0
06 Dec 2017,30.6
08 Dec 2017,0.0
16 Jan 2018,0.0
19 Jan 2018,0.0


In [15]:
## calculations

valor_hora = int(media_salarial) / 22 / 8
ammount = round(sum(over_df['overtime'])/60 * valor_hora, 2)

count = len(data_frame)

In [17]:
#Resultados
with open("resultados.txt", "w") as fl:
    print("\n", file=fl)
    print("==============", file=fl)
    print("**RESULTADOS**", file=fl)
    print("==============", file=fl)
    print("Informações coletadas de",count, "e-mails", file=fl)
    print("Data base do período prescrito:", dia_db,"/", mes_db,"/", ano_base, file=fl)
    print("Média salarial =", media_salarial, file=fl)
    print("Valor da hora Trabalhada =", valor_hora, file=fl)
    print("==============", file=fl)
    print("Total de Horas Extras =", round(sum(over_df['overtime'])), "horas", file=fl)
    print("Total devido = R$", ammount, file=fl)
    print("==============", file=fl)
    print(" ", file=fl)
    print("Valores da Hora Extra: Intrajornada = 50%  |  Noturna = 50%  |  Normal = 70%  |" \
          "Fim de Semana = 100%  |  Feriados = 100%", file=fl)

print("Tudo pronto!")
input('APERTE ENTER PARA FINALIZAR')

Tudo pronto!
APERTE ENTER PARA FINALIZAR


''